# Parse LC databases

This script tests on how to parse the databases for cams and images.



In [2]:
import pandas

## Read Cam and LiDAR database


In [15]:
#dataset_path = '/mnt/data/datasets/Spherical/test_training/'
dataset_path = '/tmp/training/'
lidar_ts_db_path = dataset_path + 'lidar_ts.csv'
cam_ts_db_path = dataset_path + 'cam_ts.csv'
missions_db_path = dataset_path + 'missions.csv'

print(f'Reading lidar db from {lidar_ts_db_path}')
print(f'Reading cam db from {cam_ts_db_path}')
print(f'Reading cam db from {missions_db_path}')

lidar_df = pandas.read_csv(lidar_ts_db_path, names=['mission_from', 'ts_from', 'mission_to', 'ts_to'], delimiter=';', header=None)
cam_df = pandas.read_csv(cam_ts_db_path, names=['mission_from', 'ts_from', 'mission_to', 'ts_to'], delimiter=';', header=None)
missions_df = pandas.read_csv(missions_db_path, names=['mission_anchor', 'mission_positive', 'mission_negative'], delimiter=',', comment='#', header=None)

print(f'\nRead {lidar_df.size} and {cam_df.size} entries.')
print(missions_df)

Reading lidar db from /tmp/training/lidar_ts.csv
Reading cam db from /tmp/training/cam_ts.csv
Reading cam db from /tmp/training/missions.csv

Read 9092 and 9092 entries.
    mission_anchor mission_positive mission_negative
0       8d1b..0000       174e..0000       8d1b..0000
1       8d1b..0000       174e..0000       8d1b..0000
2       8d1b..0000       174e..0000       8d1b..0000
3       8d1b..0000       b52f..0000       8d1b..0000
4       8d1b..0000       b52f..0000       8d1b..0000
..             ...              ...              ...
153     73cc..0000       73cc..0000       73cc..0000
154     73cc..0000       73cc..0000       73cc..0000
155     73cc..0000       73cc..0000       73cc..0000
156     73cc..0000       73cc..0000       73cc..0000
157     73cc..0000       73cc..0000       73cc..0000

[158 rows x 3 columns]


In [18]:
training_missions = ['8d1b..0000', '25b1..0000', 'ef8b..0000', 'b03a..0000', '0167..0000', '472b..0000', '0282..0000', 'e2da..0000', '8a4a..0000', '657d..0000', 'f760..0000', '73cc..0000', '0569..0000', '174e..0000', 'b52f..0000']
test_missions = ['89de..0000', '96af..0000', 'd530..0000', 'd662..0000', '62d2..0000', '6fec..0000', 'd778..0000']

In [52]:
training_indices_lidar = pandas.DataFrame()
training_indices_cam = pandas.DataFrame()

#for i in range(0, len(training_missions)):
for i in range(0, 1):
    print(f'Processing training mission: {training_missions[i]}')
    current_lidar_df = missions_df[missions_df['mission_anchor'] == training_missions[i]]    
    print(current_lidar_df['mission_positive'])
    mask = [False] * current_lidar_df['mission_positive'].size
    for j in range(0, len(training_missions)):
        mask = mask | current_lidar_df['mission_positive'].str.contains(training_missions[j]).values
    
    current_lidar_df = current_lidar_df[mask]        
    
    
    index_lidar_df = pandas.DataFrame({'index': current_lidar_df.index})    
    #print(f'lidar df size: {training_indices_lidar.size} and raw {index_lidar_df.size} and before {current_lidar_df.size}')
    training_indices_lidar = training_indices_lidar.append(index_lidar_df)
    #print(f'lidar after df size\n: {training_indices_lidar.size}')
    
    #current_cam_df = cam_df[cam_df['mission_from'] == training_missions[i]]
    #current_cam_df = current_cam_df[current_cam_df['mission_to'].isin(training_missions)]
    #index_cam_df = pandas.DataFrame({'index': current_cam_df.index})        
    #training_indices_cam = training_indices_cam.append(index_cam_df)
    

training_indices_lidar.drop_duplicates()
#training_indices_cam.drop_duplicates()

print(f'lidar df size: \n{training_indices_lidar}')
#print(f'cam df size: \n{training_indices_cam}')

#training_indices = training_indices_lidar.join(training_indices_cam.set_index('index'), on='index', how='inner')
#print(f'lidar df after size: {training_indices.size}')
#print(training_indices)

Processing training mission: 8d1b..0000
0      174e..0000
1      174e..0000
2      174e..0000
3      b52f..0000
4      b52f..0000
5      174e..0000
6      b52f..0000
7      b52f..0000
8      b52f..0000
9      b52f..0000
10     b52f..0000
11     b52f..0000
12     b52f..0000
13     b52f..0000
14     b52f..0000
15     b52f..0000
16     b52f..0000
17     b52f..0000
18     b52f..0000
19     96af..0000
20     96af..0000
21     d778..0000
22     8a4a..0000
23     298d..0000
24     89de..0000
25     96af..0000
26     96af..0000
49     8d1b..0000
50     8d1b..0000
51     8d1b..0000
52     8d1b..0000
53     8d1b..0000
Name: mission_positive, dtype: object
[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True False False False  True False
 False False False  True  True  True  True  True]
lidar df size: 
    index
0       0
1       1
2       2
3       3
4       4
5       5
6       6
7       7
8       8
9       9
10     10
11     11
12  

In [6]:
test_indices_lidar = pandas.DataFrame()
test_indices_cam = pandas.DataFrame()
    
for i in range(0, len(test_missions)):
    print(f'Processing test mission: {test_missions[i]}')
    current_lidar_df = lidar_df[lidar_df['mission_from'] == test_missions[i]]
    current_lidar_df = current_lidar_df[current_lidar_df['mission_to'].isin(test_missions)]
    index_lidar_df = pandas.DataFrame({'index': current_lidar_df.index})    
    test_indices_lidar = test_indices_lidar.append(index_lidar_df)
    
    current_cam_df = cam_df[cam_df['mission_from'] == test_missions[i]]
    current_cam_df = current_cam_df[current_cam_df['mission_to'].isin(test_missions)]
    index_cam_df = pandas.DataFrame({'index': current_cam_df.index})        
    test_indices_cam = test_indices_cam.append(index_cam_df)
    
test_indices_lidar.drop_duplicates()
test_indices_cam.drop_duplicates()
test_indices = test_indices_lidar.join(test_indices_cam.set_index('index'), on='index', how='inner')
print(test_indices)

Processing test mission: 89de..0000
Processing test mission: 96af..0000
Processing test mission: d530..0000
Processing test mission: d662..0000
Processing test mission: 62d2..0000
Processing test mission: 6fec..0000
Processing test mission: d778..0000
    index
1      10
11    139
12    159
13    185
14    213
15    241
0       9
1      14
3      24
5      33
7      42
9      56
11     70
13     85
15    101
17    117
19    134
30    268
40    433
4     138
5     158
6     184
7     212
8     240
9     269


In [31]:
cross_join = test_indices.join(training_indices.set_index('index'), on='index', how='inner')
assert(cross_join.size == 0)



NameError: name 'test_indices' is not defined